In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib as plt
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

from tensorflow import keras
#import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from keras import backend as K
import gensim

/home/medic/python36/lib64/python3.6/site-packages/OpenSSL/crypto.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import utils, x509
[nltk_data] Downloading package punkt to /home/medic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
test_data = pd.read_excel('./test_hotels.xlsx')
train_data = pd.read_excel('./train_hotels.xlsx')

In [3]:
train_data.head()

,sentiment,text
0,4,"Очень достойный отель с прекрасными номерами, ..."
1,4,"Остановились в Барселоне проездом, т.к. нужно ..."
2,4,Типичная сетевая гостиница. Главный плюс-шикар...
3,1,"Начнем с того, что в этом отеле не берут деньг..."
4,5,"Отель находится в отдалении от центра,но пешко..."


In [4]:
def create_sentiment(x):
    sentiment = 0
    if x > 3:
        sentiment = 1
    else:
        sentiment = 0
    
    return sentiment

In [5]:
# create class from digital order
train_data['class'] = train_data.apply(lambda x: create_sentiment(x['sentiment']), axis = 1)
test_data['class'] = test_data.apply(lambda x: create_sentiment(x['sentiment']), axis = 1)
# clear Nan data in datasets
train_data.dropna(subset = ['text'], inplace = True)
test_data.dropna(subset = ['text'], inplace = True)

In [6]:
train_data.head(15)

,sentiment,text,class
0,4,"Очень достойный отель с прекрасными номерами, ...",1
1,4,"Остановились в Барселоне проездом, т.к. нужно ...",1
2,4,Типичная сетевая гостиница. Главный плюс-шикар...,1
3,1,"Начнем с того, что в этом отеле не берут деньг...",0
4,5,"Отель находится в отдалении от центра,но пешко...",1
5,5,Приехали с сестрой и её мужем на машине и с со...,1
6,5,Чистота и удобство в номерах – по высшему клас...,1
7,5,В сети отелей NH Collection отдыхаем впервые. ...,1
8,5,"У отеля неплохое расположение, рядом есть разл...",1
9,3,отель соответствует заявленным звездам.номера ...,0


In [9]:
# preproccesing a data 
stopwords = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def clean_text(text):
    text = str(text)
    text = text.lower()
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwords]
    return " ".join(text)

train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)

In [11]:
train_data.head(6)

,sentiment,text,class
0,4,"достойный отель прекрасный номерами, хороший и...",1
1,4,"остановиться барселона проездом, т.к. посетить...",1
2,4,типичный сетевой гостиница. главный плюс-шикар...,1
3,1,"начать того, отель брать деньга воздух. звонок...",0
4,5,"отель находиться отдаление центра,ный пешком д...",1
5,5,приехать сестра муж машина собакой. парковка п...,1


In [12]:
train_data['class'].value_counts()

1    41507
0     8668
Name: class, dtype: int64

У нас дисбаланс классов. Это не есть хорошо.

In [13]:
train_corpus = " ".join(train_data["text"])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [14]:
max_words = 200
max_len = 40

tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:15]

['отель',
 'номер',
 'завтрак',
 'хороший',
 'персонал',
 'минута',
 'вид',
 'метро',
 'отличный',
 'расположение',
 'большой',
 'центр',
 'находиться',
 'ресторан',
 'бассейн']

In [15]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [16]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [17]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_data["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_data["text"]], dtype=np.int32)

In [18]:
x_train.shape, x_test.shape

((50175, 40), (6876, 40))

In [19]:
x_train[13]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   4,   1,   2,  33,  10,
        28,   6,   6,   8,  73,  49,   6, 116, 113,  44, 104,   1, 108,
         8], dtype=int32)

In [20]:
num_classes = 2
y_train = keras.utils.to_categorical(train_data['class'], num_classes)
y_test = keras.utils.to_categorical(test_data['class'], num_classes)

In [21]:
# model with random weights 
epochs = 40
batch_size = 512
print_batch_n = 100

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [22]:
# for calculate f1 metric it that we have disbalance classes
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [23]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = [f1])

In [133]:
%%time
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/40
89/89 [==============================] - 6s 67ms/step - loss: 0.3287 - f1: 0.8623 - val_loss: 0.3802 - val_f1: 0.8381
CPU times: user 15.6 s, sys: 279 ms, total: 15.9 s
Wall time: 6.06 s


In [134]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test f1:', score[1])

14/14 [==============================] - 0s 14ms/step - loss: 0.3297 - f1: 0.8580


Test f1: 0.8579697012901306


#### Предобученная модель. Проверим насколько лучше будут результаты у предварительно обученной модели.



In [72]:
# get list of words for search it in pretrained model 
voc = list(vocabulary.keys())
voc_new = []
for w in voc:
    w += '_NOUN'
    voc_new.append(w)

In [26]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('./220/model.bin', binary = True)
#word_vectors_matrix = [word_vectors[i][:128] for i in range(200)] 
#initializer = tf.keras.initializers.Constant(word_vectors_matrix)

In [126]:
#word_vectors.get_vector('приятный'.split('_')[0])
matrix_voc = [word_vectors[val][:128] for key, val in word_vectors.key_to_index.items()
              if key.split('_')[0] in list(vocabulary.keys())]
initializer = tf.keras.initializers.Constant(matrix_voc[:200])

In [128]:
pretren_model = Sequential()
pretren_model.add(Embedding(input_dim=max_words, embeddings_initializer=initializer,
                            output_dim=128, input_length=max_len, trainable=False))
pretren_model.add(Conv1D(128, 3))
pretren_model.add(Activation("relu"))
pretren_model.add(GlobalMaxPool1D())
pretren_model.add(Dense(10))
pretren_model.add(Activation("relu"))
pretren_model.add(Dense(num_classes))
pretren_model.add(Activation('softmax'))

In [129]:
pretren_model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = [f1])

In [132]:
%%time
history = pretren_model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/40
89/89 [==============================] - 4s 48ms/step - loss: 0.3527 - f1: 0.8507 - val_loss: 0.4124 - val_f1: 0.8230
CPU times: user 12.1 s, sys: 234 ms, total: 12.3 s
Wall time: 4.32 s


In [131]:
score = pretren_model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test f1:', score[1])

14/14 [==============================] - 0s 14ms/step - loss: 0.3773 - f1: 0.8391


Test f1: 0.8390928506851196


#### Выводы по данному исследованию:
1. Модель со свободными весами показала хорошие результаты. Точность классификации 0.8579 на нестовой выборке.
2. Модель с весами из предобученной модели показала схожие результаты, однако скорость обучения в предобученной модели выше чем у модели со свободными весами. 
Если бы наша модель была сложнее и имела больше данных то разница в скорости выполения была бы еще больше.

<table>
<thead>
    <tr><th>Model</th><th>F1 (accuracy)</th><th>Time execution</th></tr>
</thead>
<tbody>
    <tr><td>Free weights</td><td>0.8579</td><td>total: 15.9 s</td></tr>
    <tr><td>Pretrained</td><td>0.8390</td><td>total: 12.3 s</td></tr>
</tbody>
</table>